# 1. Aquisicao e tratamento dos dados

## Importar bibliotecas

In [48]:
! pip install xlrd
! pip install openpyxl
! pip install pydantic

  Using cached openpyxl-3.0.9-py2.py3-none-any.whl (242 kB)
  Using cached et_xmlfile-1.1.0-py3-none-any.whl (4.7 kB)


In [42]:
import pandas as pd
import os
import yfinance as yf

from pydantic import BaseModel
from typing import Optional, List

In [ ]:
class AquisicaoDadosFundamentos(BaseModel):
    balancos_dir: Optional[str] = "../dados/balancos/"
    fund_by_code: Optional[dict] = {}
    codes: List[str] = []

    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.codes = self.get_code_list()

    def run(self):
        self.get_balancos_by_code()
        self.get_dre_by_code()
        return self.fund_by_code
    
    def get_code_list(self):
        return [file.replace("balanco_", "").replace(".xls", "") for file in os.listdir(self.balancos_dir)]

    def get_balancos_by_code(self) -> None:

        files = os.listdir(self.balancos_dir)

        for file in files:
            code = file.replace("balanco_", "").replace(".xls", "")
            print(code)
            balanco = pd.read_excel(f"{self.balancos_dir}{file}", sheet_name=0)
            # colocar codigo na posicao 0, 0
            balanco.iloc[0, 0] = code
            # mudar coluna
            balanco.columns = balanco.iloc[0]
            balanco = balanco[1:]
            # tornar a 1ª coluna (que agora tem o nome da emrpesa)
            balanco = balanco.set_index(code)
            self.fund_by_code[code] = balanco

    def get_dre_by_code(self) -> None:

        files = os.listdir(self.balancos_dir)
        for file in files:
            code = file.replace("balanco_", "").replace(".xls", "")
            dre = pd.read_excel(f"{self.balancos_dir}{file}", sheet_name=1)
            # na primeira coluna colocar o título com o nome da empresa
            dre.iloc[0, 0] = code
            # pegar 1ª linha e tornar um cabeçalho
            dre.columns = dre.iloc[0]
            dre = dre[1:]
            # tornar a 1ª coluna (que agora tem o nome da emrpesa)
            dre = dre.set_index(code)
            self.fund_by_code[code] = self.fund_by_code[code].append(dre)

    


ad = AquisicaoDadosFundamentos()
fundamentos_by_code = ad.run()


In [54]:
class AquisicaoDadosCotacoes(BaseModel):
    cotacoes_file: Optional[str] = "../dados/cotacoes/Cotacoes.xlsx"
    codes: List[str] = []
    cotacoes: Optional[dict] = {}

    def run(self):
        self.get_cotacoes_from_excel()
        self.tratar_dados_nulos()
        return self.cotacoes

    def get_cotacoes_from_excel(self) -> None:
        """
        Le planilha com cotacoes historicas das acos do IBOV
        """
        cotacoes_df = pd.read_excel(self.cotacoes_file)

        for empresa in cotacoes_df['Empresa'].unique():
            self.cotacoes[empresa] = cotacoes_df.loc[cotacoes_df['Empresa']==empresa, :]
        
        print(len(self.cotacoes))

    def tratar_dados_nulos(self) -> None:
        """
        Remover empresas com cotacao contendo dados nulos
        """
        empresas_a_remover = []
        for empresa in self.codes:
            if self.cotacoes[empresa].isnull().values.any():
                self.cotacoes.pop(empresa)
                empresas_a_remover.append(empresa)
        self.codes = list(self.cotacoes.keys())
        print(len(self.codes))

ad_cotacoes = AquisicaoDadosCotacoes(codes=ad.codes)
cotacoes_by_code = ad_cotacoes.run()


77
65


In [61]:
# remove fundamentos das empresas que tenham cotacoes com dados nulos
codes_to_be_removed_from_fund = list(set(fundamentos_by_code.keys()) ^ set(cotacoes_by_code.keys()))
for code in codes_to_be_removed_from_fund:
    fundamentos_by_code.pop(code)

if cotacoes_by_code.keys() == fundamentos_by_code.keys():
    print("Fundamentos com cotacoes com itens nulos removidos com sucesso")
    print(len(fundamentos_by_code.keys()))


Fundamentos com cotacoes com itens nulos removidos com sucesso
65


In [66]:
def juntar_fundamentos_com_cotacoes():
    """
    Trata os data frames de fundamentos e junta as cotacoes por trimestre
    """
    codes = fundamentos_by_code.keys()
    for code in codes:
        if "Adj Close" not in fundamentos_by_code[code].columns:
            df = fundamentos_by_code[code].T
            df.index = pd.to_datetime(df.index, format="%d/%m/%Y")
            # print(df)

            # Definir data como indice e pegar somente coluna de Adj Close do df
            df_cotacao = cotacoes_by_code[code].set_index("Date")
            df_cotacao = df_cotacao[["Adj Close"]]

            # Juntar dois dataframes
            df = df.merge(df_cotacao, right_index=True, left_index=True)
            df.index.name = code

            fundamentos_by_code[code] = df
        else:
            print("Tratamento ja executado")

juntar_fundamentos_com_cotacoes()

### Ajeitando colunas

In [73]:
columns = list(fundamentos_by_code["PETR4"].columns)

def remover_empresa_colunas_diff(columns_ref: List[str]):
    """ 
    Remove empresas que nao tenham colunas de acordo com colunas de acao referencia 
    """
    codes = fundamentos_by_code.keys()
    empresa_a_remover = []
    for code in codes:
        if set(columns_ref) != set(fundamentos_by_code[code].columns):
            empresa_a_remover.append(code)
    
    for empresa in empresa_a_remover:
        fundamentos_by_code.pop(empresa)
    
    print(len(fundamentos_by_code))

remover_empresa_colunas_diff(columns)

61


In [95]:
def select_columns_for_calc(codes):
    for code in codes:
        colunas_para_calculos = ["Ativo Total", "Patrimônio Líquido", "Receita Líquida de Vendas e/ou Serviços", "Lucro/Prejuízo do Período", "Adj Close"]
        df_c = fundamentos_by_code[code][colunas_para_calculos]
        df_c = df_c.rename(columns={"Ativo Total": "ativo_total", "Patrimônio Líquido": "patrimonio_liq", "Receita Líquida de Vendas e/ou Serviços": "receita_liq", "Lucro/Prejuízo do Período": "lucro_op", "Adj Close": "cotacao"})
        df_c.iloc[:,:-1] = df_c.iloc[:,:-1].astype(float)
        fundamentos_by_code[code] = df_c

codes = fundamentos_by_code.keys()
select_columns_for_calc(codes)

In [102]:
def calcular_dupont():
    codes = fundamentos_by_code.keys()
    for code in codes:
        df = fundamentos_by_code[code]

        af = df["ativo_total"] / df["patrimonio_liq"]
        ra = df["lucro_op"] / df["ativo_total"]
        ga = df["receita_liq"] / df["ativo_total"]
        ml = df["lucro_op"] / df["receita_liq"]
        rpl = af * ra
        roe = df["lucro_op"] / df["patrimonio_liq"]
        roa = ga * ml

        df["RPL"] = rpl
        df["ROE"] = roe
        df["ROA"] = roa
        df["AF"] = af
        df["RA"] = ra
        df["GA"] = ga
        df["ML"] = ml

        fundamentos_by_code[code] = df[["RPL", "ROE", "ROA", "AF", "RA", "GA", "ML", "cotacao"]]

calcular_dupont()

In [117]:

columns = fundamentos_by_code["PETR4"].columns

valores_vazios = dict.fromkeys(columns, 0)
total_linhas = 0
empresa_with_null = []

for empresa in fundamentos_by_code:
    tabela = fundamentos_by_code[empresa]
    total_linhas += tabela.shape[0]
    for coluna in columns:
        qtde_vazios = pd.isnull(tabela[coluna]).sum()
        if qtde_vazios > 0:
            empresa_with_null.append(empresa)
        valores_vazios[coluna] += qtde_vazios

print(valores_vazios)
print(total_linhas)
print(set(empresa_with_null))

# remove empresas com valores nulos, ignorando ABEV por ter nulo na ultima linha
for empresa in set(empresa_with_null):
    if empresa != "ABEV3":
        fundamentos_by_code.pop(empresa)

print(len(fundamentos_by_code))

{'RPL': 3, 'ROE': 3, 'ROA': 35, 'AF': 1, 'RA': 3, 'GA': 3, 'ML': 3, 'cotacao': 0}
2004
{'BRAP4', 'ABEV3', 'CSAN3', 'RADL3'}
58


In [ ]:
codes = list(fundamentos_by_code.keys())
codes

## Criacao de rotulos - coluna target

In [131]:
def get_ibov_from_yahoo():
    data_inicial = "2012-12-20"
    data_final = "2021-09-20"

    ibov = yf.download("^BVSP", start=data_inicial, end=data_final)

    print(ibov)

get_ibov_from_yahoo()

[*********************100%***********************]  1 of 1 completed
                Open      High       Low     Close  Adj Close    Volume
Date                                                                   
2012-12-20   60992.0   61276.0   60622.0   61276.0    61276.0   3083400
2012-12-21   61271.0   61271.0   60221.0   61007.0    61007.0   3157400
2012-12-26   61003.0   61322.0   60861.0   60960.0    60960.0   2598000
2012-12-27   60964.0   61191.0   60233.0   60416.0    60416.0   2852600
2012-12-28   60416.0   61066.0   60416.0   60952.0    60952.0   2626000
...              ...       ...       ...       ...        ...       ...
2021-09-13  114300.0  117046.0  114300.0  116404.0   116404.0   9253400
2021-09-14  116405.0  117270.0  115809.0  116181.0   116181.0   9034100
2021-09-15  116191.0  116312.0  114741.0  115063.0   115063.0   9456800
2021-09-16  115062.0  115062.0  113395.0  113794.0   113794.0  10685500
2021-09-17  113794.0  113794.0  111157.0  111439.0   111439.0  1482